## Loading Data

In [6]:
import spacy
import random
import operator
import re
from collections import defaultdict
import copy
from collections import Counter
import string
unlp = spacy.blank('ur')

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Loading the whole data which acts as the corpus.
2. Stored in Lines as sentences

In [8]:
with open('/content/drive/MyDrive/NLP/data.txt','r') as f:
  Lines = f.readlines()

print(len(Lines))

4652818


1. Removing all the \n characters from the end of sentences
2. Data list now contains all the tokenized words instead of sentences.

In [ ]:
data = []

for j in range(len(Lines)):

  words = Lines[j].split(' ')
  clean = []

  # Removing the \n from the last word
  for i in words:
    clean.append(i.strip())
  
  for i in clean:
    data.append(i)


data

No \n characters are present anymore

In [10]:
print(data[0:20])



['sai', 'kha', 'ya', 'her', 'kisi', 'kay', 'bus', 'ki', 'bat', 'nhi', 'hai', 'lakin', 'main', 'ki', 'hal', 'kal', 'bi', 'aj', 'aur', 'aj']


Loading mispelled data

In [11]:
with open('/content/drive/MyDrive/NLP/NLP4_i18-0562/misspellings.txt','r') as f:
  m_data = f.readlines()

print(len(m_data))

36101


Storing all the correct words and their possible mistypes in the m_dict dictionary.
1. The key is the correct word example 'ka'
2. The values are then basically the list of mistyped words.

Example: 
{'ka': ['kaz', 'cka', 'mka', 'kga', 'yka', 'kba']}

In [ ]:
m_dict = {}

for j in range(len(m_data)):

  if j == 0:
    continue

  clean = []
  r_spaces = " ".join(m_data[j].split())
  words = r_spaces.split(' ')
  words[0] = words[0][:-1]

  for i in range(len(words)):
    if i==0:
      continue
    clean.append(words[i])

  m_dict[words[0]] = clean
  

print(m_dict)

Listing words for demonstartion of 'ka'

In [13]:
print(m_dict['ka'])


['kaz', 'cka', 'mka', 'kga', 'yka', 'kba']


##Word Unigram Model

Returns the probabilties of all the unigrams. Also coded using dictionaries so very optimised on such a large data.

From Our research paper.
Each candidate correction, c, is scored by
Pr(c) Pr(tlc), and then normalized by the sum
of the scores for all proposed candidates. The
prior, Pr(c), is estimated by
(freq(c) + 0.5)/N, where freq(c) is the
number of times that the word c appears in the
1988 AP corpus (N = 44 million words))

1. Here in the unigram model(prior) ,0.5 is added to the unigram occurences.
2. This change has been catered in the code


In [14]:
def Unigram_Probabilities(total):

  u_dict = {}
  
  for i in total: #Iterate over the whole tokenized list

    if i in u_dict:   # If word is present in dictionary keys already add a +1
      u_dict[i]+=1
    else:
      u_dict[i] = 1   # This means key is being put in the first time

  for key in u_dict:  # Dividing all by total vacablury for bigram formula
    u_dict[key] = float((u_dict[key]+0.5)/len(total))  # Adding 0.5 as showed in research paper

  return u_dict # Reurning the dictionary



In [15]:
unigram_probs =  Unigram_Probabilities(data)


##Character unigram and bigram model

Creating a tokenized list of alphabets from the file.
1. '#' character is added before each word here.
2. '#' is later critical for use in the discontiious function of error model calculation

In [17]:
char_data = []
for i in Lines:
  for j in i:
    if j == '\n':     
      continue
    char_data.append(j)
    if j == ' ':
      char_data.append('#')


Calculating probabilities for singel character unigram.
1. Will be used in error model later on.
2. Does not contain probabilty of \n
3. Used specificaly for calculation in insertion and substituition functions in error model

In [18]:
def charUnigramProbabilities(total):

  u_dict = {}
  
  for i in total: #Iterate over the whole tokenized list

    if i in u_dict:   # If word is present in dictionary keys already add a +1
      u_dict[i]+=1
    else:
      u_dict[i] = 1   # This means key is being put in the first time

  for key in u_dict:  # Dividing all by total vacablury for bigram formula
  
    u_dict[key] = float((u_dict[key])/len(total))  # Adding 0.5 as showed in research paper

  return u_dict # Reurning the dictionary

In [19]:
char_unigram_prob = charUnigramProbabilities(char_data)

In [20]:
char_unigram_prob['b']

0.016241374495379317

Bigram Probabilties for characters
1. Used for the calculation of delete and transpose functions in error model.
2. Plays role as the denominator.
3. Probabilty of each bigram of letters is divided by the total occurences of the first letter in bigram.

  1. For this the unigram character model above is used.
  2. Dictionaries use for optimisation 

In [ ]:
def Bigram_Probabilty(total):

  mainDict = {}  

  for i in range(len(total)-1):

    if (total[i],total[i+1]) in mainDict:  # If bigram is in dict,increment
       mainDict[total[i],total[i+1]] += 1
    else:
      mainDict [total[i],total[i+1]] = 1   # If not initialise it with 1

  #divide by unigram model
  for g in mainDict:
    mainDict[g] /= char_unigram_prob[g[0]]

  return mainDict

bigram_prob = Bigram_Probabilty(char_data)
bigram_prob

## Create insert, delete, substitution and transposition tables using misspellings

1. Making 2D confusion matrixes for insert,delete,reverse and substitute using python dictionaries
2. The insert Matrix has been displayed for demonstration
3. For start and $ for end have also been catered in the matrices

In [ ]:
# Dictionaries for error model
d1 = dict.fromkeys(string.ascii_lowercase, 0)
d2 = dict.fromkeys(string.ascii_lowercase, 0)
d3 = dict.fromkeys(string.ascii_lowercase, 0)
d4 = dict.fromkeys(string.ascii_lowercase, 0)

# INSERT TABLE
insert = dict.fromkeys(string.ascii_lowercase, 0)
insert['#'] = copy.deepcopy(d1)   # For start Character
for a in insert:
  for b in d1:
    insert[a] = copy.deepcopy(d1)
insert['$'] = copy.deepcopy(d1)   # For end character


# DELETE TABLE
delete = dict.fromkeys(string.ascii_lowercase, 0)
delete['#'] = copy.deepcopy(d2)   # For start Character
for a in delete:
  for b in d2:
    delete[a] = copy.deepcopy(d2)
delete['$'] = copy.deepcopy(d2)   # For end character


# Transpose TABLE
transpose = dict.fromkeys(string.ascii_lowercase, 0)
transpose['#'] = copy.deepcopy(d3)   # For start character
for a in transpose:
  for b in d3:
    transpose[a] = copy.deepcopy(d3)
transpose['$'] = copy.deepcopy(d3)   # For end character


# SUBSTITUTE TABLE
substitute = dict.fromkeys(string.ascii_lowercase, 0)
substitute['#'] = copy.deepcopy(d4)   # For start character
for a in substitute:
  for b in d4:
    substitute[a] = copy.deepcopy(d4)
substitute['$'] = copy.deepcopy(d4)   # For end character

# FOR DEMONSTRATION
print("Insert Matrix")

# insert['a']['b'] += 5 # Adding a value to show access

for a in insert:
  print(a,"->",insert[a])


ERROR MODEL CLASS

1. This class will basically hold the error table probabilties.
2. It has the 4 different matrices of insert,transpose ,delete and substitute.
3. On calling the constructor the fillMatrices function is called autmatically using the dictionary of misspelled words.


fillMatrices Member function()

1. This functions recives a dictionary m_data.
  1. In the dictionary the key are the correct words and the corresponding ist in the values contains all the incorrect words with edit distance one.
  2. This dictionary is made from the misspeliings.txt shown above.
2. Now the important part is to increment the values and for that we first have to know wether the error word is a delete,insert,transpose or substitute and then update the corresponding value of letters in the x and y axis of the respective table.

3. Insert:
  1. If the length of error word is greater then correct word then it means that an extra word was inserted and hence we move into the insert if condition.
  2. In Insert there are basically 3 condition to look for.
  3. If the first alphabets dont match then increment the first character with # in x axis and the mismached alphabet of the wrong word.
  4. Finally the mismatch could also accour somewhere in between. This is detected by the loop and in case of mismathch the isert table is incremented.

4. Delete:
  Delete is very similar to insert.
  1. The if condition here is that the error word is less then the original word.
  2. The three condition futher are then checked accordingly and the letters are aupdated in the delete table.

5. Transpose and Substituition
  1. These two operationactually kind of overlap and can be found in a very interesting way.
  2. The lengths of both words have to be equal in this case and so the words are compared with each other in a loop. If the letters at the same index mismatch and the next alpahbet in eqalu to the mismatched alphabet of the forst word then it means that the word woas a transpose.Also the fist letter of the second matches the second letter of the fisr word.In this case the transpose table is updated at those letters.
  3. However if the letters mismatch and the next letters of both the words are a match or the mismatch occurs at the final letters of both words then it means that the this is a substituition and the substitute table in the error model class is updated accordingly. 


In [59]:
class Error_Model:

  def __init__(self,a,b,c,d,m):

    self.I = copy.deepcopy(a)
    self.D = copy.deepcopy(b)
    self.T = copy.deepcopy(c)
    self.S = copy.deepcopy(d)

          # Function Call to fill matrices
    self.fillConfusionMatrices(m)


  def fillConfusionMatrices(self,misspellings):

  # i is the correct word
  # j is the incorrect word

    for i in misspellings:

      L = misspellings[i]

      for j in L:    # Iterating over the list of incorrect words for the correct word i
        if len(j) > len(i):  # Insert Case
          
          if j[0] != i[0]:        # First Character Different
            self.I['#'][j[0]]+=1

          elif j[-1] != i[-1]:    # Last Character Different
            self.I[j[-2]][j[-1]]+=1

          else:                   # Middle Character Different
            for k in range(len(i)):
              if j[k] == i[k]:
                continue
              if j[k] != i[k]:
                self.I[j[k-1]][j[k]]+=1
       
        elif len(j) < len(i): # Delete case

          if j[0] != i[0]:        # First Character Different
            self.D['#'][i[0]] += 1

          elif j[-1] != i[-1]:    # Last Character Different
            self.D[i[-2]][i[-1]] += 1

          else:                   # Middle Character Different
            for k in range(len(j)):
              if j[k] == i[k]:
                continue
              if j[k] != i[k]:
                self.D[i[k-1]][i[k]] += 1

        # Transpose and substituition case
        else:
          for k in range(len(j)):
            if j[k] == i[k]:
              continue
            elif i[k] != j[k]:         
              if (k != (len(i)-1)):  #End condition
                # Transpose
                if i[k] == j[k+1] and i[k+1] == j[k]:    
                  self.T[i[k]][i[k+1]] += 1
                # Substitute
                else:
                  self.S[i[k]][j[k]] += 1
              # Last unequal alphabet so substitute
              else:
                self.S[i[k]][j[k]] += 1


1. Making an error model object providing it with empty matrices and the misspellings dictionary.
2. The object will reciave these matrices and then populate them by default as the fill matrices function is called in the constructor.

In [60]:
e_model =Error_Model(insert,delete,transpose,substitute,m_dict)

In [ ]:
# FOR DEMONSTRATION
print("Insert Matrix")
for a in e_model.I:
  print(a,"->",e_model.I[a])

In [ ]:
# FOR DEMONSTRATION
print("Delete Matrix")
for a in e_model.D:
  print(a,"->",e_model.D[a])

In [ ]:
# FOR DEMONSTRATION
print("Transpose Matrix")
for a in e_model.T:
  print(a,"->",e_model.T[a])

In [ ]:
# FOR DEMONSTRATION
print("Substitute Matrix")
for a in e_model.S:
  print(a,"->",e_model.S[a])

## Creating function that calculates P(x|w) using the Error Model tables.

errorModelProb Function.

1. This is the most important function of the noisy channel model as this is basically the discontiniuos function that checks which table the candidate word belongs to and then return the value of the letter changed probability from the required matrix.

2. The letters to choose as the x axis and then the y axis are determined for the unique formule of each table form the discontinious function given.

3. In Insert and substitute table the char unigram model is used in the denominator.

4. In the delete and transpose table the bigram char model is used in the denominator.

5. Checking which table the candidate belong to:

  1. If length of candidate is less then error word then insert table.
  2. If length of candidate is greate then error word then delete table.
  3. If length of candidate word is equal then check for transpose in pairs of two.If the first pair in each word does not match and the pairs word match in cross method then use the transpose table.
  4. If transpose is not true and two letters do not match each other but the letter after them do match then use the substituition table.

In [65]:
def errorModelProb(error_word,candidate,char_bigram,char_unigram,error_model):

  # j is error word


  ########### FOR INSERT

  if len(error_word) > len(candidate):
    if error_word[0] != candidate[0]:   # First Character Different
      return (error_model.I['#'][error_word[0]])/1

    elif error_word[-1] != candidate[-1]:    # Last Character Different
      return (error_model.I[candidate[-2]][error_word[-1]])/char_unigram[candidate[-2]]

    else:                   # Middle Character Different
      for k in range(len(candidate)):
        if error_word[k] == candidate[k]:
          continue
        if error_word[k] != candidate[k]:
          return (error_model.I[candidate[k-1]][error_word[k]])/char_unigram[candidate[k-1]]


    ########### FOR DELETE
  elif len(error_word) < len(candidate):
    if error_word[0] != candidate[0]:   # First Character Different
          return (error_model.D['#'][candidate[0]])/1

    elif error_word[-1] != candidate[-1]:    # Last Character Different
      return (error_model.D[candidate[-2]][candidate[-1]])/char_bigram[(candidate[-2],candidate[-1])]


    else:                   # Middle Character Different
      for k in range(len(error_word)):
        if candidate[k] == error_word[k]:
          continue
        if candidate[k] != error_word[k]:
          return (error_model.D[candidate[k-1]][candidate[k]])/char_bigram[(candidate[k-1],candidate[k])]


    ########### FOR DELETE AND TRANSPOSE

  else:
    for k in range(len(candidate)):
      if candidate[k] == error_word[k]:
        continue
      elif candidate[k] != error_word[k]:         
        if (k != (len(candidate)-1)):  #End condition
          # Transpose
          if candidate[k] == error_word[k+1] and candidate[k+1] == error_word[k]:    
            return error_model.T[candidate[k]][candidate[k+1]] / char_bigram[(candidate[k],candidate[k+1])]

          # Substitute
          else:        
            return error_model.S[error_word[k]][candidate[k]] / char_unigram[candidate[k]]

          # Last unequal alphabet so substitute
        else:
            return error_model.S[error_word[k]][candidate[k]] / char_unigram[candidate[k]]

Demonstrating noisy channel prob with Insert case

In [66]:
errorModelProb('baba','abba',bigram_prob,char_unigram_prob,e_model)

2.022408952886818e-06

Demonstrating noisy channel prob with Delete case

In [67]:
errorModelProb('m','um',bigram_prob,char_unigram_prob,e_model)

8.0

Demonstrating noisy channel prob with Transpose case

In [68]:
errorModelProb('tmu','tum',bigram_prob,char_unigram_prob,e_model)

3.907554898871633e-07

Demonstrating noisy channel prob with Substitute case

In [69]:
errorModelProb('gmum','ghum',bigram_prob,char_unigram_prob,e_model)

1485.3580332494614

##Generating Candidate Words

For this assignment due to lack of time on my part I have used the inbuilt function of the DamerauLevenshtein distance which also checks the transpose and gives the value of transpose  and substitute as also 1.

For all the other parts of assignment no inbuit functions or libraries such as NLTK are used. Everything else is made using native data structures and are explained how they are used in text boxes.

In [70]:
pip install pyxDamerauLevenshtein

Show 1 distance for transpose.

In [71]:
from pyxdameraulevenshtein import damerau_levenshtein_distance

damerau_levenshtein_distance('ka','ak')

1

Generate Candidate Words()
Logic:
1. Since the words have to be from the vocablury, all our unique words are already present in the dictianary that holds the unigram probabilties.
2. Iterate over the keys of this dictionary and compare the edit distance of each key word with the error word.
3. If distance is equal to 1 then add it the word to list of candidates.
4. As a result all the candidate words will be a subset of the vocablury as required by the assignment.

In [72]:
def generateCandidateWords(error_word):

  candidates = []   # List of candidate words

  for i in unigram_probs: # Iterating unique words form vocablury

    count = damerau_levenshtein_distance(error_word,i)

    if count == 1:
      candidates.append(i)

  return candidates



Generating candidate words for a demo word for demonstration

In [ ]:
generateCandidateWords('baba')

##Function for finding w^ from a list of candidate words,error model and unigram model

correctedWord()

1. This function recieves the list of candidate words,error model class and the word unigram model.
2. After recieving these first there are checks to see if the candidate words are present or not and there checks are present.
  
  1. If empty display empty message and return.
  2. If not empty display the list and proceed further. 

3. errorModelProb() which is basically the discontinious function for error probabilty finds out which table the candidate word belongs too and returns the noisy model probability.
4. Once this probabilty is returned ,it is multiplied with the unigram probabilty of that candidate word.
5. This multiplied probabilty is stored in the dictionary words with the candidate as the key and its probabibilty(( P(x|w) * P(w))) as the value.
6. Similarly if the prob of the word is higher then the current max prob then the value of max prob is updated.
7. Finally when all the candidate word probabilties have been calculated and stored the dictionary is looped over and the keys with the prob = max_prob are stored in max words list.
8. This max_words list conatianng the one or more highest prob words are then displayed.

In [74]:
def correctedWord(error,candidates,error_model,unigram_probs):

  max = -1;
  words = {}

  if len(candidates) == 0:
    print("No Candidate Words so no corrections for this word !!")
    return

  print("Candidate Words->")
  print(candidates,"\n")

  for i in candidates:

    # This part demonstrates the ( P(x|w) * P(w))
    words[i] =  errorModelProb(error,i,bigram_prob,char_unigram_prob,error_model)* unigram_probs[i]

    # Storing max probability
    if words[i] > max:
      max = words[i]

  # For finding out argmax amongst all the candidates
  max_words =[]
  for j in words:
    if words[j] == max:
      max_words.append(j)

  print("Corrected Words with highest Probability")
  print(max_words)

  return

SCENARIO 1: There are possible candidate words of a error word.

Here the error word is 'bataoe'

The candidate words are : ['batao', 'batae', 'bataye', 'bataon', 'batane', 'batate', 'bataoge']

The most corrected word from our noisy channel model is ['batao']

In [75]:
e_word = 'bataoe'
print("Error Word-> ",e_word,'\n')

candidates = generateCandidateWords(e_word)

correctedWord(e_word,candidates,e_model,unigram_probs)

Error Word->  bataoe 

Candidate Words->
['batao', 'batae', 'bataye', 'bataon', 'batane', 'batate', 'bataoge'] 

Corrected Words with highest Probability
['batao']


SCENARIO 2: There are no candidate words for the word hathoraet.

A message has been printed telling that no correct assumption can be made since no candidate words were found. 

Caters to point 6 of the assignment where are function should handle this case as well.

In [76]:
e_word = 'hathoraet'
print("Error Word-> ",e_word,'\n')

candidates = generateCandidateWords(e_word)

correctedWord(e_word,candidates,e_model,unigram_probs)

Error Word->  hathoraet 

No Candidate Words so no corrections for this word !!
